# GPU

In [3]:
!nvidia-smi

Thu May  5 20:39:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.23       Driver Version: 511.23       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:08:00.0  On |                  N/A |
| 41%   55C    P5    28W / 310W |   1485MiB /  8192MiB |     33%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### 1. 计算设备

In [6]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

我们可以查询可用gpu的数量。

In [7]:
torch.cuda.device_count()

1

In [9]:
def try_gpu(i=0):  #@save
    """如果存在,则返回gpu(i),否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU,如果没有GPU,则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

### 2. 张量与GPU

我们可以查询张量所在的设备。 默认情况下，张量是在CPU上创建的。

In [10]:
x = torch.tensor([1,2,3])
x.device

device(type='cpu')

#### 2.1. 存储在GPU上

In [15]:
X = torch.ones([2,3],device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

### 3. 神经网络与GPU

In [18]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [19]:
net(X)

tensor([[-0.1029],
        [-0.1029]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [22]:
net[0].weight.device

device(type='cuda', index=0)